In [1]:
print('Cosmic Ray Analysis Code by Shawn - main function outline:', end = '\n 1 ')
print('Import data file', end = '\n 2 ')
print('Modify data - rescale, detrend, remove re-zeros, etc.', end = '\n 3 ')
print('Matched filter peak finder', end = '\n 4 ')
print('Initial analysis tools - spectrum analyzer, etc.', end = '\n 5 ')
print('Fitter and fit rejection', end = '\n 6 ')
print('Coincidence analysis', end = '\n 7 ')
print('Plot')

Cosmic Ray Analysis Code by Shawn - main function outline:
 1 Import data file
 2 Modify data - rescale, detrend, remove re-zeros, etc.
 3 Matched filter peak finder
 4 Initial analysis tools - spectrum analyzer, etc.
 5 Fitter and fit rejection
 6 Coincidence analysis
 7 Plot


In [2]:
print('Imports')
import matplotlib
matplotlib.use('TkAgg')
import pylab as pl
#from DFT import *
import numpy as np
import numpy.polynomial.polynomial as poly
from scipy import signal
from scipy import optimize
from scipy import math
from scipy.fftpack import rfft, irfft, fftfreq, ifft, fft
from scipy.signal import argrelextrema
from scipy.signal import find_peaks_cwt
from scipy.optimize import curve_fit
import scipy.integrate as integrate
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import shutil
import statistics
from statistics import median_low
from matplotlib import cm
from matplotlib import mlab
from math import exp, expm1
import sys
import bisect
import os
import gc
import pickle

Imports


In [3]:
print('Defining things')
V2uA_SQ1 = 25.2 #define SQUID calibrations in uA/V
V2uA_SQ2 = 25.2
V2uA_SQ3 = 30.0
V2uA_SQ4 = 30.0
V2uA_SQ5 = 25.9
V2uA_SQ6 = 25.2

xaxes = []
ymeansubs = []
timeconsts = []
energyhists = []
rates = []
allphases = []
maxvalsxs = []
maxvalsys = []

Defining things


In [4]:
print('Supporting functions')

def extract_raw_data(fn,colNames):
    '''
    Extract txt data from a LabVIEW output file fn and return
    a Python dictionary. colNames is a list of strings
    corresponding to each column in the txt file.

    d['filename'] returns the filename.
    d[column name] returns an array with the data from that column.
    '''
    f = open(fn)
    d = {}
    d['filename'] = fn
    llist = []
    for i in range(len(colNames)):
        llist.append([])
    for line in f:
        lineSplit = line.split()
        for i in range(len(colNames)):
            llist[i].append(float(lineSplit[i].replace(',','')))
    for i in range(len(colNames)):
        d[colNames[i]] = pl.array(llist[i])
    f.close()
    return d


###########################################################################


def extract_raw_ts(fn):
    '''
    Extract XY data from a LabVIEW output file fn and
    return a Python dictionary with entries x and y.

    d['filename'] returns the filename.
    '''
    colNames = ['y0','y1','y2','y3','y4','y5','y6','y7']
    #colNames = ['y0','y1']
    return extract_raw_data(fn,colNames)

def extract_raw_ts_3ch(fn):
    colNames = ['y0','y1', 'y2']
    return extract_raw_data(fn,colNames)

def extract_raw_ts_1ch(fn):
    colNames = ['y0']
    return extract_raw_data(fn,colNames)


###########################################################################


def template(x):
    xsc = 100.
    relsc = 100./7.
    phs = 0.4975
    if(x<0.5): 
        return 1/(1+math.exp(-xsc*relsc*(x-phs)))
    else:
        return math.exp(-xsc*(x-0.499848))

def filtertempl(x):
    return 1 - math.exp(-(10.*x))


###########################################################################


def CR_combine(fnList,newfn):
    with open(newfn + '.dat','wb') as wfd:
        for f in fnList:
            with open(f,'rb') as fd:
                shutil.copyfileobj(fd, wfd, 1024*1024*10)


###########################################################################


def find_nearest(array, value):
    array = np.asarray(array)
    minarr = array - value
    idx = np.argmin(i for i in minarr if i > 0)
    idxx = (np.abs(array - value)).argmin()

    return array[idxx].tolist()

def find_nearest_fix(array, value):
    array = np.asarray(array)
    minarr = array - value
    idx = np.argmin(i for i in minarr if i > 0)
    idxx = (np.abs(array - value)).argmin()

    return array[idx].tolist()



Supporting functions


In [5]:
print('Main function')


def CR_analyze(data, sets, SQs, gain, bias, scantime, inv, pklsuffix, savefits = True, savefigs = True, showfits=False, fluxfix = True, noisefilt = False, coincidence = False, verbose=True, detrend=True, pwrcnv = False, fit = True):


    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # Parameter input explanations:
    # data: The name of the data file, up to the number. Should always end in "00x.txt", so this input is "dataname_" and the "00x.txt" is added to the string based on the 'sets' input
    # sets: A two value array of format [beginning data set, end data set]
    # SQs: Right now this is set up for three column inputs, so the inupt is of format [1,2,3] if the data was being taken with SQs 1, 2, and 3 for example
    # gain: An array of the channel gains used in the power conversion
    # bias: An array of biases used
    # scantime: Length of each data set in seconds
    # noisefilt: A work in progress function to remove certain noise components using a notch fitler applied to the power spectra
    # verbose: Outputs more information about jumped flux removal and eventually fits
    # detrend: Detrends each timestream
    # inv: Inverts the data. Right now we'll need to know before running if the peaks need to be inverted for this input. Will change this to detect automatically in the future.
    # pwrcnv: Converts y to units of power
    # fit: Option to fit the data
    

#Example parameter inputs: CR_analyze(["20210319_SQ1-contr1_SQ2-contr2_SQ6-contr3_biasthreeturnsintotrans_5-FTR-5min_Co60_"], [5,10], [1,2,6], [100.,100.,100.], [1.,1.,1.], 300., detrend=True, inv = True, pwrcnv = True)
    
    # note: will add input for number of channels in the future
    
    # Defining datasets to look at
   
    pl.clf()
    datalist = []
    jfe_datasets = []
    energies = [[],[],[]]
    energies_data = [[],[],[]]
    energies_nojfes = [[],[],[]]
    timeconsts = [[],[],[]]
    rates = [[],[],[]]
    peakvals = [[],[],[]]
    errors = []
    #meantimeconsts = [[],[],[]]
    e_tot = [] #[[datalists][channels][energies]]
    tau_tot = [] #[[datalists][channels][tau]]
    rt_tot = [] #[[datalists][channels][risetimes]]
    coinc_tot = [] #[[datalists][x value]
    
    
    if len(sets)==1:
        if sets[0] < 10:
            dl = data[0] + "00" + str(sets[0]) + ".txt"
        if sets[0] >= 10 and sets[0] < 100:
            dl = data[0] + "0" + str(sets[0]) + ".txt"
        if sets[0] >= 100:
            dl = data[0] + str(sets[0]) + ".txt"
        datalist.append(dl)
    else:
        setnumb = len(range(sets[0], sets[1]+1))
        sn=[]
        for i in range(setnumb):
            sn.append(sets[0]+i) # Make an array that starts with the first dataset number you input
            if sn[i] < 10:
                dl = data[0] + "00" + str(sn[i]) + ".txt"
            if sn[i] >= 10 and i < 100:
                dl = data[0] + "0" + str(sn[i]) + ".txt"
            if sn[i] >= 100:
                dl = data[0] + str(sn[i]) + ".txt"
            datalist.append(dl)
            e_tot.append(sn[i]) # Count up from first dataset
            tau_tot.append(sn[i])
            rt_tot.append(sn[i])
            coinc_tot.append(sn[i])
            
    #finding the writepath
    numslashes = 9
    slashes = []
    for i in range(len(data[0])):
        if data[0][i] =='/':
            slashes.append(0)
        if len(slashes) == numslashes:
            writepath = data[0][:i+1]
            break
    wpsuffix = pklsuffix[0]
            
     
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    
    # Start of primary for loop
    
    for n in range(len(datalist)): 
        if verbose:
            print(datalist[n])   
        print("running data set", datalist[n][-7:-4])
        dList0 = []
        jfe_channels = [] #jumped flux events
        
        
        d = extract_raw_ts_3ch(datalist[n])
        dList0.append(d)
        #print(len(dList0))
        #loop for each channel in the data set
        ch = []
        ymeansub = []
        
        notchyms = []
        maxvalsy = []
        maxvalsx = []
        #print("assign SQ cals")
        for j in range(len(SQs)): 
            ymeansub_disc = []
            c = "y"+ str(j)
            ch.append(c)
            print("channel", SQs[j])
            #e_tot[n].append(j) # Append for each channel
            #tau_tot[n].append(j)
            #rt_tot[n].append(j)
            #coinc_tot[n].append(j)
            
            #Assign SQ calibrations
            if SQs[j]==1:
                V2uA0 = V2uA_SQ1
            else:
                if SQs[j]==2:
                    V2uA0 = V2uA_SQ2
                else:
                    if SQs[j]==3:
                        V2uA0 = V2uA_SQ3
                    else:
                        if SQs[j]==4:
                            V2uA0 = V2uA_SQ4
                        else:
                            if SQs[j]==5:
                                V2uA0 = V2uA_SQ5
                            else:
                                if SQs[j]==6:
                                    V2uA0 = V2uA_SQ6
            
            #print("define lists")
            if pwrcnv:
                yList0 = (10**6)*(10**-5)*bias[j]*V2uA0*dList0[0][ch[j]]/gain[j] 
            else:
                yList0 = dList0[0][ch[j]]    

            # Define samplerate
            samplerate = len(yList0)/scantime
            
            # 'Normalize' x
            xList0 = range(0,len(yList0))
            xNorm0 = (scantime/len(xList0))
            xLn0 = []
            for i in range(len(xList0)):
                normd = xNorm0*xList0[i]
                xLn0.append(normd)
                
            #print("start fluxfix")
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
            
            
    # Fixing jumped flux
            
            # Finding discontinuities 
            deryList0 = np.gradient(yList0) # Take the first derivative of the y-data
            discthresh = 50*np.std(deryList0)
            #discthresh = 115*np.std(deryList0) # Set the threshold for peakfinding, 20 standard deviations works well
            maxsx = []
            maxsy = []
            xdisc_N = []
            
            
            for i in range(len(yList0)):
                if abs(deryList0[i]) > discthresh and abs(np.mean(yList0[i - int(samplerate*0.1):i])-np.mean(yList0[i:i + int(samplerate*0.1)])) > discthresh:
                #if abs(np.mean(yList0[:i])-np.mean(yList0[i:])) > discthresh:
                    # Set the condition that the peaks found in the derivitive are above the descrimination threshold
                        # and also that the means of the data before and after the found peak are under a similar threshold.
                    # This distinguishes hits from jumped flux.
                    maxsx.append(xLn0[i])
                    maxsy.append(deryList0[i]) # Make a list of values above the threshold
            
            if len(maxsy) > 0: # If discontinuities are found 
                maxtime = maxsx[maxsy.index(max(maxsy))]
                roundedx = np.round(maxsx, 2) # Round the x values to find general locations
                xdisc = np.unique(roundedx) # Insert rounded x values into their own array, N_array = N_disc.
                roundxmain = np.array(np.round(xLn0, 2)) # Round all x data
                #pl.plot(xdisc, [0,0], 'o',color='C2')
                
                
                # Removing discontinuities
                
                remove = int(samplerate * 0.015) # Number of points to remove before disc location
                remove_last = int(samplerate * 0.01) # Number of point to remove after disc location. Charactaristic of most jumped flux recover times - would like to make mroe robust in the future
                ind1 = []
                ind2 = []
                
                for i in range(len(xdisc)): # Loop over each found discontinuity
                    xdisc_N.append(np.where(roundxmain == xdisc[i])[0][0]) # Find index of x locations
                    
                    if len(xdisc) == 1: # If there's only one discontinuity detected
                        ymeansub_disc.extend(yList0[:xdisc_N[i]] - np.mean(yList0[:xdisc_N[i]])) # Remove DC offset before disc.
                        ymeansub_disc.extend(yList0[xdisc_N[i]:] - np.mean(yList0[xdisc_N[i]:])) # Remove DC offset after disc.
                    else:
                        if i == 0: # Remove DC offset before first disc.
                            ymeansub_disc.extend(yList0[:xdisc_N[i]] - np.mean(yList0[:xdisc_N[i]]))
                        if i != len(xdisc)-1 and i != 0: # Remove DC offset after last disc., before next
                            ymeansub_disc.extend(yList0[xdisc_N[i-1]:xdisc_N[i]] - np.mean(yList0[xdisc_N[i-1]:xdisc_N[i]]))
                        if i == len(xdisc)-1: # Remove DC offset after the last disc.
                            ymeansub_disc.extend(yList0[xdisc_N[i-1]:xdisc_N[i]] - np.mean(yList0[xdisc_N[i-1]:xdisc_N[i]]))
                            ymeansub_disc.extend(yList0[xdisc_N[i]:] - np.mean(yList0[xdisc_N[i]:]))
                    
                    # Replace datapoints where disc. occurs with mean values
                    ind1.append(xdisc_N[i]-remove)
                    ind2.append(xdisc_N[i]+remove_last)
                    #ind2.append(xdisc_N[i]+remove)
                
                
                for i in range(len(ind1)):
                    ymeansub_disc[ind1[i]:ind2[i]] = [np.mean(ymeansub_disc)]*len(ymeansub_disc[ind1[i]:ind2[i]])
                if fluxfix:
                    yList0 = ymeansub_disc # Overwrite y data
                
            
            # Some print options for jumped flux info
            if verbose: 
                if len(maxsy) == 0:
                    print("No jumped flux events found in channel %i, dataset "%j + datalist[n][-7:-3])
                    jfe_channels.extend([0])
                if len(maxsy) > 0:
                    if len(xdisc) == 1:
                        print("One jumped flux event found in channel %i, dataset "%j + datalist[n][-7:-3])
                        jfe_channels.extend([1])
                if len(maxsy) > 0:
                    if len(xdisc) > 1:
                        print("%i "%len(xdisc) + "jumped flux events found in channel %i, dataset "%j + datalist[n][-7:-3])
                        jfe_channels.extend([len(xdisc)])
            else:
                if len(maxsy) == 0:
                    jfe_channels.extend([0])
                if len(maxsy) > 0:
                    if len(xdisc) >= 1:
                        jfe_channels.extend([len(xdisc)])
                        
                      
            
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

            
    # Remaining dataset processing
            #print("end fluxfix")
            # Detrend sets with no jumped flux
            if detrend:
                yList0det = signal.detrend(yList0) #detrend raw data
            else:
                yList0det = yList0
            
            # Invert
            if inv[j]:
                for i in range(len(yList0det)):
                    yList0det[i] = -1.*yList0det[i]

            # First DC offset removal before finding noise threshold
           
            if len(maxsy) == 0:
                ymeansub.append(yList0det - np.mean(yList0det))
            else:
                ymeansub.append(yList0det)
            
                      
      
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # Noise filtering
            
            if noisefilt:
                fft_freq, fft_psd = signal.welch(ymeansub[j], samplerate)
                b, a = signal.iirnotch(215, 30, samplerate)
                notchymssub = signal.lfilter(b, a, ymeansub[j], axis=- 1, zi=None)
                
                #hpfilt100 = signal.butter(1, 300, 'lp', output = 'sos', fs=5000)
                #filtered100 = signal.sosfilt(hpfilt100, ymeansub[j])
                
                hpfilt100 = signal.butter(1, 100, 'hp', output = 'sos', fs=5000)
                filtered100 = signal.sosfilt(hpfilt100, ymeansub[j])
                
                hpfilt20 = signal.butter(1, 20, 'hp', output = 'sos', fs=5000)
                filtered20 = signal.sosfilt(hpfilt20, ymeansub[j])
                filtered20_0_float = signal.sosfilt(hpfilt20, ymeansub[j])
                filtered20_1_HS = signal.sosfilt(hpfilt20, ymeansub[j])
                filtered20_0_float_source = signal.sosfilt(hpfilt20, ymeansub[j])
                filtered20_1_HS_source = signal.sosfilt(hpfilt20, ymeansub[j])
                
                #file1 = open("testfile.txt", "a")
                #for s in range(len(filtered20)):
                    #print(str(filtered20))
                    #sys.stdout.write(str(filtered20[s]))
                #sys.stdout.write(str(filtered20))
                
                
                #if j == 1:
                pl.plot(xLn0, ymeansub[j])
                pl.show()
                pl.clf() 
                if n==0 and j == 0:
                    filtered20_0_ns = signal.sosfilt(hpfilt20, ymeansub[j])
                
                if n==0 and j == 1:
                    filtered20_1_ns = signal.sosfilt(hpfilt20, ymeansub[j])
                    
                if n==0 and j == 2:
                    filtered20_2_ns = signal.sosfilt(hpfilt20, ymeansub[j])
                    
                if n==1 and j == 0:
                    filtered20_0_s = signal.sosfilt(hpfilt20, ymeansub[j])
                
                if n==1 and j == 1:
                    filtered20_1_s = signal.sosfilt(hpfilt20, ymeansub[j])
                    
                if n==1 and j == 2:
                    filtered20_2_s = signal.sosfilt(hpfilt20, ymeansub[j])

                    #pl.title("Background vs. Source, Control bolo, Floating, 20 Hz HPF, bolo 1")
                    #n1, bins1, patchs1 = pl.hist(filtered20_0_ns, bins='auto', alpha = 0.75, label = 'Control bolo - no source')
                    #n2, bins2, patchs2 = pl.hist(filtered20_0_s, bins='auto', alpha = 0.5, label = 'Control bolo - Co-60')
                    #pl.legend(loc='upper right')
                    #pl.axvline(x=0.0, color='r', linestyle='--', alpha = 0.5)
                    #pl.show()
                    #pl.clf()
                    
                    #pl.title("Background vs. Source, Control bolo, Floating, 20 Hz HPF, bolo 2")
                    #n1, bins1, patchs1 = pl.hist(filtered20_1_ns, bins='auto', alpha = 0.75, label = 'Control bolo - no source')
                    #n2, bins2, patchs2 = pl.hist(filtered20_1_s, bins='auto', alpha = 0.5, label = 'Control bolo - Co-60')
                    #pl.legend(loc='upper right')
                    #pl.axvline(x=0.0, color='r', linestyle='--', alpha = 0.5)
                    #pl.show()
                    #pl.clf()
                    
                    #pl.title("Background vs. Source, Control bolo, Floating, 20 Hz HPF, bolo 3")
                    #n1, bins1, patchs1 = pl.hist(filtered20_2_ns, bins='auto', alpha = 0.75, label = 'Control bolo - no source')
                    #n2, bins2, patchs2 = pl.hist(filtered20_2_s, bins='auto', alpha = 0.5, label = 'Control bolo - Co-60')
                    #pl.legend(loc='upper right')
                    #pl.axvline(x=0.0, color='r', linestyle='--', alpha = 0.5)
                    #pl.show()
                    #pl.clf()
                    
                #if n==0 and j == 2:
                    #filtered20_2_ns = signal.sosfilt(hpfilt20, ymeansub[j])

                    #pl.title("Energy, Control vs. Paladium, Floating, Co-60, 20 Hz HPF")
                    #n1, bins1, patchs1 = pl.hist(filtered20_0_ns, bins='auto', alpha = 0.75, label = 'Control bolo')
                    #n2, bins2, patchs2 = pl.hist(filtered20_2_ns, bins='auto', alpha = 0.75, label = 'Paladium mitigation')
                    #pl.legend(loc='upper right')
                    #pl.axvline(x=0.0, color='r', linestyle='--', alpha = 0.5)
                    #pl.show()
                    #pl.clf()
                #if j ==1:
                    #pl.title("Energy, Floating vs. Heatsunk, Control bolo, 20 Hz HPF, no source")
                    #n1, bins1, patchs1 = pl.hist(filtered20_1_ns, bins='auto', alpha = 0.75, label = 'Control bolo - heat sunk')
                    #n2, bins2, patchs2 = pl.hist(filtered20_0_ns, bins='auto', alpha = 0.75, label = 'Control bolo - floating')
                    #pl.legend(loc='upper right')
                    #pl.axvline(x=0.0, color='r', linestyle='--', alpha = 0.5)
                    #pl.show()
                    #pl.clf()
                
                
            
                
                notchyms.append(notchymssub)
                fft_freq2, fft_psd2 = signal.welch(notchyms[j], samplerate)
                #pl.plot(fft_freq2, fft_psd2)
                #pl.clf()
                #pl.plot(fft_freq, fft_psd)
                #pl.show()
                #pl.clf()
                #if n == 1:
                #pl.title("Timestream, raw (blue), 20 Hz/50 ms hpf (green), 100 Hz/10 ms  hpf (red) ")
            #pl.title("Timestream, raw (blue), 300 Hz  lpf (red) ")
                #pl.plot(xLn0, ymeansub[j])
            #pl.plot(xLn0, filtered20, 'g')
                #pl.plot(xLn0, filtered20, 'r')
                #pl.show()
                #pl.clf()
                #pl.hist(ymeansub[j], bins='auto', label = "unfiltered")
                #pl.hist(filtered20, bins = 'auto', label = "20 Hz/50 ms hpf")
                #print(filtered20)
                #pl.hist(filtered100, bins = 'auto', label = "300 Hz lpf")
                #pl.legend(loc='upper right')
                #pl.show()
                pl.clf() 
                
            
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

            
             
        
    # Peak finder
            
            if fit:
                # Setup matched filter peak finder
                xmf = np.arange(0.4, 0.6, 0.0001) #define window/xaxis for convolving function
                ymf = []
                for k in range(len(xmf)):
                    ymf.append(template(xmf[k])) # Create matched filter convolving function

                #Convolve templates with data, take derivative to find rise location, find max
                pf = signal.fftconvolve(ymeansub[j], ymf, mode='same')
                fftpf = rfft(pf)
                yderiv = np.gradient(pf)
                yderivdetr = signal.detrend(yderiv)
                #ydstd = 0.02
                peakthresh = 8.0
                ydstd = peakthresh*np.std(yderivdetr) # Seems to be a good threshold based on the noise of the data set and goodness of fits
                #ydstd = peakthresh*np.std(yderivdetr) # Seems to be a good threshold based on the noise of the data set and goodness of fits
                ydpks = []
                xydpks = []
                yderivdetr_temp = []
                ymeansub_temp = []
                indextemp = []
               
                #removing peaks from timestreams to get an accurate std of the noise
                for i in range(len(yderiv)):
                    
                    if i>25 and 0.75*yderivdetr[i-25] > ydstd or yderivdetr[i-25] < -0.75*ydstd: #numbers tweaked to optimize peak removal
                        yderivdetr_temp.append(0)
                        ymeansub_temp.append(0)
                        indextemp.append(i)
                        
                    else:
                        yderivdetr_temp.append(yderivdetr[i])
                        ymeansub_temp.append(ymeansub[j][i])
                
                stddata = 3*np.std(ymeansub_temp)
                ydstdtemp = []
                stdtemp = []
                #finding peaks using new std      
                for i in range(len(yderiv)):
                    ydstdtemp.append(ydstd)
                    stdtemp.append(stddata)
                    if yderivdetr[i] > stddata:
                        ydpks.append(yderivdetr[i])
                        xydpks.append(xLn0[i])
                        
                        
                # Final DC offset removal using peak-removed data
                ymeansub[j] = ymeansub[j] - np.mean(ymeansub_temp) 
                
                
                
                #print(indextemp)
                #for i in range(len(indextemp)):
                    #ymeansub_temp[indextemp[i]-5000:indextemp[i]+5000] == 0
                    
                        
                
                
                #pl.clf()   
                #pl.plot(xLn0, yderivdetr)
                #pl.plot(xLn0, yderivdetr_temp)
                #pl.plot(xLn0, ydstdtemp, 'r')
                #pl.plot(xLn0, stdtemp, 'b')
                #pl.plot(xLn0, ymeansub[j])
                #pl.plot(xLn0, ymeansub_temp)
                #pl.show()
                #pl.clf()
                
               
                # Pick out max
                mvalsx_init = []
                mvalsy_init = []
                mvalsx = []
                mvalsy = []
                for i in range(len(ydpks)):
                    if (i+1) >= len(ydpks):
                        break
                    #print(xydpks[i+1]-xydpks[i])
                    if (ydpks[i+1]<ydpks[i]) and (ydpks[i-1]<ydpks[i]):
                        mvalsy_init.append(ydpks[i])
                        mvalsx_init.append(xydpks[i])
                #print(mvalsx_init)
                
                # remove duplicate peaks that occur within 5 ms of each other
                for i in range(len(mvalsx_init)):
                    if (i+1) >= len(mvalsx_init):
                        mvalsx.append(mvalsx_init[i])
                        mvalsy.append(mvalsy_init[i])
                        break
                    if mvalsx_init[i+1] - mvalsx_init[i] > 0.005:
                        mvalsx.append(mvalsx_init[i])
                        mvalsy.append(mvalsy_init[i])
                #print(mvalsx)
                
                maxvalsx.append(mvalsx)
                maxvalsy.append(mvalsy)
                
                #rate = len(maxvalsy)/scantime
                #print(maxvalsx[j], maxvalsy[j])
                if len(maxvalsx[j]) > 0:
                    rates[j].append(len(maxvalsx[j]))
                else:
                    rates[j].append(0)
                    
                    
                
                
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    
    # Start fitting 
    
            # Start fit for exponential tail
                #print("JFE length:")
                #print(jfe_channels)
                #print(len(jfechannels[i])
               #print("Event numbers:")
               # print(len(maxvalsx[j]))
                # Define pulse windows
                #pl.plot(xLn0, ymeansub[j])
                #jfe_list = [] #makes a list of JFEs of the same length as events
                #if jfe_channels[i] == 0 and len(maxvalsx[j]) > 0:
                    #jfe_list.append(0)
               # if jfe_channels[i] != 0 and len(maxvalsx[j]) > 0:
                    #jfe_list.append(1)
            
                #print(jfe_list)
               # print(len(jfe_list))
               # print(maxvalsx[j])
               # print(len(maxvalsx[j]))
                
                if len(maxvalsx[j]) > 0:
                    
                    fitlistsx = []
                    fitlistsx_sc = []
                    fitlistsy = []
                    datarange = int(0.02*samplerate)
                    #datarange = int(0.035*samplerate) # Defines the window of each pulse fit
                    fitlistsx_re = []
                    fitlistsx_sc_re = []
                    fitlistsy_re = []
                    fitlistsx_p = []
                    fitlistsy_p = []
                    fitlistsx_sc_p = []
                    #datarange_re = int(0.1*samplerate)
                    datarange_re = int(0.1*samplerate) 
                    #datarange_p = int(0.004*samplerate)
                    #datarange_p = int(0.01*samplerate)
                    datarange_p = int(0.005*samplerate)
                    
                    def glitchfit(x, a, b): #defining a second degree polynomial to fit the event data in log space
                        return a + x**b
                    #datarange = int(0.015*samplerate) # Defines the window of each pulse fit
                    #datarange = int(0.035*samplerate)
                    #datarange_ext = int(0.04*samplerate)
                    #datarange_ext = int(0.1*samplerate)
                    
                    
                    for i in range(len(maxvalsx[j])):
                        
                        #newjfelist = []
                        #if jfelist[i]
                            #newjfelist.append()
                        
                        fitlistx = []
                        fitlisty = []
                        fitlistx_sc = []
                        flaggedfits = []
                        #taus = []
                        #energies = []
                        threesigmas = []
                        fntointmods = []
                        phases = []
                        maxs = []
                        polyorder = 12
                        
                        
                        
                        #e_tot[n][j].append() # Defne zero from beginning mean (defined in rising edge), subtract each point from zero point, add up until mean is reached again after peak
                        #tau_tot[n][j].append() # Time from max of the peak fit to one e-fold of the fit
                        #rt_tot[n][j].append() # Time from rise start to max of the peak fit
                        #coinc_tot[n][j].append() #
                        
                        
                        
                        for l in range(datarange):
                            fitlistx.append(maxvalsx[j][i]+(float(l)/samplerate)) # For each event, add an x data point until the end of the defined window
                            fitlistx_sc.append(int(samplerate*maxvalsx[j][i])+(l)) # Do the same but in units of samples in order to define y-index
                            fitlisty = ymeansub[j][fitlistx_sc[0]:fitlistx_sc[-1]+1] # Define y window
                            
                        fitlistsx.append(fitlistx)
                        fitlistsx_sc.append(fitlistx_sc)
                        fitlistsy.append(fitlisty)


                    # Start of fitter for exponential tail
                    # Here we will define the pulse window, then move to log space, fit a first degree polynomial, then convert back
                        phase = fitlistsx[i][np.argmax(fitlistsy[i])] # Defines the max of the y fit data's x location
                        
                        fitlistx_chop = fitlistsx[i][np.argmax(fitlistsy[i]):] # Selecting only the tails for fitting in log space
                        fitlisty_chop = fitlistsy[i][np.argmax(fitlistsy[i]):]
                        
                        # This chunk of code is to remove the negatives before going to log space
                        # I decided to remove the nans from log space in case it affected the fit later
                        #fitlisty_chop_zeros = np.where(fitlisty_chop > 0, fitlisty_chop, 0*-fitlisty_chop) # Setting all negative values to zero 
                        #fitlisty_chop_zeros_logic = np.logical_not(fitlisty_chop_zeros) # Logic array where 0 = True
                        #fitlisty_chop_nonzeros_logic = np.logical_not(fitlisty_chop_zeros_logic) # Inverse logic array
                        #fitlisty_chop_mod = fitlisty_chop[fitlisty_chop_nonzeros_logic] # Making array for interpolate function with zeros removed
                        #interp_y = np.interp(fitlisty_chop_zeros_logic.nonzero()[0], fitlisty_chop_nonzeros_logic.nonzero()[0], fitlisty_chop_mod) # Interpolating function
                        #fitlisty_chop[fitlisty_chop_zeros_logic] = interp_y # Setting the indexed values of the zero points to the interpolated function's values
                        
                        np.seterr(invalid='ignore') # Ignore nan error from negative values in log
                        fitlistx_logchop = np.log(fitlistx_chop)  # Take the log of each event for fitting
                        fitlisty_logchop = np.log(fitlisty_chop)
                        
                        # Remove nan values from the data and interpolate between them for fits
                        nan_indices = np.isnan(fitlisty_logchop) # Logic array where nan = True
                        real_indices = np.logical_not(nan_indices) # Inverse logic array
                        real_data = fitlisty_logchop[real_indices] # Making array for interpolate function with nans removed
                        if len(real_data) == 0:
                            print("error in fit, moving to next")
                            errors.append(1)
                            break
                        
                        interp_y = np.interp(nan_indices.nonzero()[0], real_indices.nonzero()[0], real_data) # Interpolate function
                        fitlisty_logchop[nan_indices] = interp_y # Setting the indexed values of the zero points to the interpolated function's values
            
                        #npts = 18
                        #npts = len(fitlistsx[i][:np.argmax(fitlistsy[i])]) # Set th number of points before the peak
                        #xlogfit = np.log(fitlistsx_chop)
                        
                        # This is a seed fit to provide seed values for a higher order fit
                        coefs = poly.polyfit(fitlistx_logchop, fitlisty_logchop, 2) # Define a polynomial to fit the events in log space
                        
                        #print(coefs) # Flag fits that have a positive slope
                        #if coefs[1] > 0:
                            #flaggedfits.append()
                            #continue

                        ffit = poly.polyval(fitlistx_logchop, coefs) # Creating the fit in log space
                        
                        xfit = np.exp(fitlistx_logchop) # Convert back from log
                        ffitexp = np.exp(ffit)
                        
                        
                        #pl.clf()
                        #pl.plot(np.exp(fitlistx_logchop), np.exp(fitlisty_logchop))
                        #pl.plot(fitlistx_logchop, ffit)
                        
                        #pl.plot(xLn0, yList0det)
                        #pl.plot(xfit, ffitexp)
                        #pl.plot(fitlistsx[i][np.argmax(fitlistsy[i])], fitlistsy[i][np.argmax(fitlistsy[i])], 'o')
                        #pl.show()
                        
                        
                        #creating paramerst find bad fits - checks to see if the time constant is negative, then re-fits exp with 1st degree poly instead of 2nd
                        #maxvalexp = np.max(ffitexp)
                        #maxvalexpindex = ffitexp.tolist().index(maxvalexp)
                        #maxvalexpindex = np.where(ffitexp==maxvalexp)
                        #onesigmaexp = maxvalexp*(1/np.exp(1))
                        #sigmaminexpx = xfit[ffitexp.tolist().index(find_nearest(ffitexp, onesigmaexp))]
                        #sigmaminexpy = ffitexp[ffitexp.tolist().index(find_nearest(ffitexp, onesigmaexp))]
                        #thistauexp = 1*(sigmaminexpx - xfit[maxvalexpindex])
                       # if thistauexp < 0.0:
                           # print("asdf1")
                           # coefs = []
                            #ffit = []
                           ## ffitexp = []
                            #coefs = poly.polyfit(fitlistx_logchop, fitlisty_logchop, 1)
                           # ffit = poly.polyval(fitlistx_logchop, coefs)
                            #ffitexp = np.exp(ffit)
                            #ffitexp_totalmod = ffitexp_totapmod_new
                        
                        
                        # This is the primary fit (still in log space) that uses values from the seed fit for a more accurate fit
                        # Modeled with a second degree polynoial with seeds from the first degree fit in log space
                        # params, params_covariance = optimize.curve_fit(glitchfit, fitlistx_logchop, fitlisty_logchop, p0 = [coefs[1], coefs[0], -1.], bounds = ((-np.inf,-np.inf,-np.inf),(0.,np.inf,0.)))
                        # glitchfitlog = glitchfit(fitlistx_logchop, params[0], params[1], params[2])
                        
                        # To 0th order correct fit overshoot at start of exp. curve
                        ffitexpmod = []
                        for k in range(len(ffitexp)):
                            if ffitexp[k] > np.max(fitlistsy[i]):
                                ffitexpmod.append(np.max(fitlistsy[i]))
                            else:
                                ffitexpmod.append(ffitexp[k])
                                
                        #pl.figure(1)
                        #pl.plot(fitlistx_logchop, fitlisty_logchop)
                        #pl.plot(fitlistx_logchop, ffit)
                        #pl.show()

    
        # Start fit for rising edge
        
                    # Define pulse windows, '_re' = 'rising edge'
                    
                    
                        fitlistx_re = []
                        fitlisty_re = []
                        fitlistx_sc_re = []
                        #xshift = 50
                        #xshift = 58
                        #xshift = 10
                        xshift = 10
                        for l in range(datarange_re):
                            fitlistx_re.append(maxvalsx[j][i] + (float(l)/samplerate) - (xshift/samplerate)) # For each event, subtract an x data point until the end of the defined window. Adding more points at the beginning to catch rising edge
                            fitlistx_sc_re.append(int(samplerate*maxvalsx[j][i])+(l) - xshift) # Do the same but in units of samples in order to define y-index
                            fitlisty_re = ymeansub[j][fitlistx_sc_re[0]:fitlistx_sc_re[-1]+1] # Define y window
                            
                        #print(fitlistx_sc_re[0])
                        #print(fitlistx_sc_re[-1]+1)

                        fitlistsx_re.append(fitlistx_re)
                        fitlistsx_sc_re.append(fitlistx_sc_re)
                        fitlistsy_re.append(fitlisty_re)
                        
                        
                            
                        
                        #pl.plot(fitlistsx_re[i],fitlistsy_re[i])
                        #pl.show()
                        
                        
                        
                        # Start of fitter for rising edge
                        # Here we'll define the rising edge's window and fit a line to it without going into log space
                        
                        if len(fitlistsy_re[i]) == 0: #accounts for error in very few data sets where fitlistys_re[i] is not populated
                            print("defined window invalid (676)")
                            errors.append(1)
                            continue
                        phase = fitlistsx_re[i][np.argmax(fitlistsy_re[i])] # Defines the max of the y fit data's x location
                        
                        fitlistx_re_chop = fitlistsx_re[i][:np.argmax(fitlistsy_re[i])] # Selecting only the rising edges
                        fitlisty_re_chop = fitlistsy_re[i][:np.argmax(fitlistsy_re[i])]
                        
                        startrise = []
                        meany = []
                        fitlisty_re_chop_temp = []
                        for l in range(len(fitlistx_re_chop)):
                            if (l+1) >= len(fitlistx_re_chop):
                                break
                            #print("check")
                            fitlisty_re_chop_temp.append(fitlisty_re_chop[l])
                            meany = 0.
                            meanystd = 0.
                            meany = np.mean(fitlisty_re_chop_temp)
                            meanystd = np.std(fitlisty_re_chop_temp)
                            #print(meanys, meanystd)
                            #print(fitlisty_re_chop[l] - meany, meanystd)
                            if l > 5 and fitlisty_re_chop[l] - meany > 2*meanystd: # Get data for a few points first, then look to see if the difference from the current point to the mean is larger than 2x the standard deviation of the points 
                                rise_index = l - int(samplerate/5000)
                                #startrise.append(fitlistx_re_chop(l))
                                #print(rise_index)
                                #print("rise is at %f"%fitlistx_re_chop[l])
                                break
                            else:
                                rise_index = 0
                        
                        if rise_index == 0: #if it can't find the rising edge, lower the threshold
                            for l in range(len(fitlistx_re_chop)):
                                if (l+1) >= len(fitlistx_re_chop):
                                    break
                                if l > 5 and fitlisty_re_chop[l] - meany > 1.5*meanystd: # Get data for a few points first, then look to see if the difference from the current point to the mean is larger than 2x the standard deviation of the points 
                                    rise_index = l - int(samplerate/5000)
                                    #startrise.append(fitlistx_re_chop(l))
                                    #print(rise_index)
                                    #print("rise is at %f"%fitlistx_re_chop[l])
                                    break
                        
                        fitlistx_re_fit = fitlistx_re_chop[rise_index:]
                        fitlisty_re_fit = fitlisty_re_chop[rise_index:]
                        
                        index_last = rise_index + len(ffitexp)
                        totalfitx = fitlistx_re[rise_index:index_last]
                        totalfity = fitlisty_re[rise_index:index_last]
                        
                        
                        coefs_re = poly.polyfit(fitlistx_re_fit, fitlisty_re_fit, 3) #polyorder) # Define a second degree polynomial to fit the events
                        
                        #print(coefs_re) # Flag fits that have a negative slope
                        #if coefs_re[1] < 0:
                            #flaggedfits.append()
                            #continue

                        ffit_re = poly.polyval(fitlistx_re_fit, coefs_re) # Creating the fit
                        
                    
                        
                        
                        
                        
            # Start fit for the peak
        
                    # Define pulse windows, 'p' = 'peak'
                    
                    
                        fitlistx_p = []
                        fitlisty_p = []
                        fitlistx_sc_p = []
                        #xshift_p = -10
                        xshift_p = -5
                        for l in range(datarange_p):
                            fitlistx_p.append(maxvalsx[j][i] + (float(l)/samplerate) - (xshift_p/samplerate)) # For each event, subtract an x data point until the end of the defined window. Adding more points at the beginning to catch rising edge
                            fitlistx_sc_p.append(int(samplerate*maxvalsx[j][i])+(l) - xshift_p) # Do the same but in units of samples in order to define y-index
                            fitlisty_p = ymeansub[j][fitlistx_sc_p[0]:fitlistx_sc_p[-1]+1] # Define y window
                            
                        #print(fitlistx_sc_p[0])
                        #print(fitlistx_sc_p[-1]+1)

                        fitlistsx_p.append(fitlistx_p)
                        fitlistsx_sc_p.append(fitlistx_sc_p)
                        fitlistsy_p.append(fitlisty_p)
                        
                        
                        
                        coefs_p = poly.polyfit(fitlistx_p, fitlisty_p, 10) # Define a second degree polynomial to fit the events
                        
                        #coeffs_lag_p = np.polynomial.laguerre.Laguerre.fit(fitlistx_p, fitlisty_p, 3)
                        #ffit_lag_p = np.polynomial.laguerre.lagval(fitlistx_p, coeffs_lag_p.convert().coef)
                        #print(coefs_re) # Flag fits that have a negative slope
                        #if coefs_re[1] < 0:
                            #flaggedfits.append()
                            #continue

                        ffit_p = poly.polyval(fitlistx_p, coefs_p) # Creating the fit
                        
                        #testfit_params, paramscov = curve_fit(glitchfit, fitlistx_p, fitlisty_p)
                        #tfp0 = testfit_params[0]
                        #tfp1 = testfit_params[1]
                        
                        ########################################################
                        
                  
                        #index_last = rise_index + len(ffitexp) + 200
                        index_last = rise_index + len(ffitexp) + 100
                        totalfitx = fitlistx_re[rise_index:index_last]
                        totalfity = fitlisty_re[rise_index:index_last]
                        phase_index = np.argmax(totalfity)
                        totalfitx_re = totalfitx[:phase_index]
                        totalfitx_exp = totalfitx[phase_index:]
                        
                        alpha1 = 100*len(totalfitx)
                        alpha2 = 5*len(totalfitx)
                        alpha3 = 10*len(totalfitx)
                        phase1 = fitlistsx_re[i][np.argmax(fitlistsy_re[i])-10]
                        phase2 = fitlistsx_re[i][np.argmax(fitlistsy_re[i])+0]
                        phase3 = fitlistsx_re[i][np.argmax(fitlistsy_re[i])-10]
                        function1 = []
                        function2 = []
                        function3 = []
                        
                        #print("test")
                        
                        ffitexp_total = np.exp(poly.polyval(np.log(totalfitx), coefs))
                        ffit_p_total = poly.polyval(totalfitx, coefs_p)
                        ffitexp_totalmod = []
                        ffit_re_total = poly.polyval(totalfitx, coefs_re)
                        
                        # this loop is to fix the issue of the start of the exponential fits going way above the peak in the data
                        for k in range(len(ffitexp_total)):
                            if ffitexp_total[k] > np.max(fitlistsy[i]):
                                ffitexp_totalmod.append(np.max(fitlistsy[i]))
                            else:
                                ffitexp_totalmod.append(ffitexp_total[k])
                        
                        
                        
                        
                        #print(len(ffitexp_total, ffitexp_total))
                        ffit_re_total = poly.polyval(totalfitx, coefs_re)
                        ffitexp_total[:phase_index+3] = np.max(fitlistsy[i])
                        ffit_re_total[phase_index-3:] = np.max(fitlistsy[i])
                        
                        
                        
                        #xx = np.linspace(-1, 1, len(totalfitx))
                        xx = np.linspace(totalfitx[0], totalfitx[-1], len(totalfitx))
                        sigma1 = 1/(1+np.exp((-xx+phase1)*alpha1))
                        sigma2 = 1/(1+np.exp((-xx+phase2)*alpha2))
                        sigma3 = 1/(1+np.exp((-xx+phase3)*alpha3))
                        for l in range(len(totalfitx)): 
                            function1.append((1 - sigma1[l]) * ffit_re_total[l]+sigma1[l] * ffit_p_total[l])
                            function2.append((1 - sigma2[l]) * ffit_p_total[l]+sigma2[l] * ffitexp_totalmod[l])
                            function3.append((1 - sigma3[l]) * ffit_re_total[l] +sigma3[l] * function2[l])
                            #function3.append((1 - sigma1[l]) * ffit_re_total[l] + sigma2[l] * ffitexp_totalmod[l])
                            
                        
                        
                        #Record all values for histogramming
                        energies[j].append(sum(function3))
                        energies_data[j].append(sum(totalfity)/len(totalfity))
                        #energies_nojfes[j].append()
                        peakvals[j].append(fitlistsx[i][np.argmax(fitlistsy[i])])
                        
                        

                        
                        
                        #define params for time constants
                        maxval = np.max(function3)
                        maxvalindex = function3.index(maxval)
                        onesigma = maxval*(1/np.exp(1))
                        sigmaminx = totalfitx[ffitexp_totalmod.index(find_nearest(ffitexp_totalmod,onesigma))]
                        sigmaminy = function3[ffitexp_totalmod.index(find_nearest(ffitexp_totalmod,onesigma))]
                        thistau = 1*(sigmaminx - totalfitx[maxvalindex])
                        
                        
                            
                        
                    
                        #timeconsts[j].append(1*(sigmaminx - totalfitx[maxvalindex]))
                        #timeconsts[j].append(thistau)
                        #print(timeconsts)
                        
                        
                        #pl.plot(sigmaminx - thistau, np.max(function3), 'o')
                        #pl.plot(sigmaminx, sigmaminy, 'o')
                        
                        #pl.plot(totalfitx, totalfity)
                        #pl.plot(totalfitx, function3, 'r')
                        #pl.plot(xfit, ffitexp, 'g')
                        
                        #pl.plot(fitlistx_p, ffit_p, 'g')
                        
                        #print(ffit_lag_p)
                        #pl.plot(fitlistx_p, ffit_lag_p, 'r')
                        #pl.plot(fitlistx_p, ffit_lag_p, 'r')
                        
                        #pl.plot(fitlistx_re_fit, ffit_re, 'r')
                        #pl.plot(xfit, ffitexp, 'r')
                        
                        gl_zeros = []
                        normxdisc = []
                        if len(xdisc_N) > 0:
                            for k in range(len(xdisc_N)):
                                #gl_zeros.append(ymeansub[j][np.where(xLn0 = xNorm0*xdisc_N[k])])
                                gl_zeros.append(discthresh)
                                normxdisc.append(xNorm0*xdisc_N[k])
                            #print("test")
                            #print(normxdisc)
                            #print(maxs)
                        
                        #pl.plot(xLn0, deryList0)
                        #pl.clf()
                        
                       
                        if thistau < 0.0:
                            #print("asdf2")
                            #pl.figure()
                            #pl.title('dataset: ' + str(datalist[n][-7:-4]) + ', channel: ' + str(SQs[j]))
                            #pl.plot(np.log(totalfitx), np.log(totalfity))
                    
                            #pl.plot(np.log(totalfitx), np.log(function3), 'r')
                            #pl.pause(0.05)
                            #pl.show(block=False)
                            #correcting, re-fitting
                            coefs = poly.polyfit(fitlistx_logchop, fitlisty_logchop, 1) # Define a polynomial to fit the events in log space
                            ffit = poly.polyval(fitlistx_logchop, coefs) # Creating the fit in log space
                            xfit = np.exp(fitlistx_logchop) # Convert back from log
                            ffitexp = np.exp(ffit)
                            function1 = []
                            function2 = []
                            function3 = []
                            ffitexp_total = np.exp(poly.polyval(np.log(totalfitx), coefs))
                            ffitexp_totalmod = []
                            # this loop is to fix the issue of the start of the exponential fits going way above the peak in the data
                            for k in range(len(ffitexp_total)):
                                if ffitexp_total[k] > np.max(fitlistsy[i]):
                                    ffitexp_totalmod.append(np.max(fitlistsy[i]))
                                else:
                                    ffitexp_totalmod.append(ffitexp_total[k])

                            ffitexp_total[:phase_index+3] = np.max(fitlistsy[i])
                            for l in range(len(totalfitx)): 
                                function1.append((1 - sigma1[l]) * ffit_re_total[l]+sigma1[l] * ffit_p_total[l])
                                function2.append((1 - sigma2[l]) * ffit_p_total[l]+sigma2[l] * ffitexp_totalmod[l])
                                function3.append((1 - sigma1[l]) * ffit_re_total[l] +sigma1[l] * function2[l])
                            print("failed fit found, correcting")
                            #pl.figure()
                            #pl.title('dataset: ' + str(datalist[n][-7:-4]) + ', channel: ' + str(SQs[j]))
                            #pl.plot(np.log(totalfitx), np.log(totalfity))
                            #pl.plot(np.log(totalfitx), np.log(function3), 'r')
                            #pl.pause(0.05)
                            #pl.show(block=False)
                            #define params for time constants
                            maxval = np.max(function3)
                            maxvalindex = function3.index(maxval)
                            onesigma = maxval*(1/np.exp(1))
                            sigmaminx = totalfitx[ffitexp_totalmod.index(find_nearest(ffitexp_totalmod,onesigma))]
                            sigmaminy = function3[ffitexp_totalmod.index(find_nearest(ffitexp_totalmod,onesigma))]
                            thistau = 1*(sigmaminx - totalfitx[maxvalindex])
                            timeconsts[j].append(thistau)
                        else:
                            timeconsts[j].append(thistau)
                        if thistau < 0.0:
                        #if coefs_p[0] < 0.0:
                            print("could not correct")
                            errors.append(1)
                        
                        
                        if showfits:
                            pl.clf()
                            pl.figure()
                            pl.title('dataset: ' + str(datalist[n][-7:-4]) + ', channel: ' + str(SQs[j]))
                            pl.plot(totalfitx, totalfity)
                            
                            #pl.plot(normxdisc, gl_zeros, 'o')
                            #pl.plot(totalfitx, ffit_p_total, 'r')
                            pl.plot(totalfitx, function3, 'r')
                            #pl.plot(totalfitx, function3, 'g')
                            pl.pause(0.05)
                            pl.show(block=False)
                        
                        if savefits:
                            #os.mkdir(writepath + 'fitplots/')
                            newdir = os.path.join(writepath+'fitplots/')
                            if not os.path.exists(newdir):
                                os.mkdir(newdir)
                
                            pl.clf()
                            pl.figure()
                            pl.title('dataset: ' + str(datalist[n][-7:-4]) + ', channel: ' + str(SQs[j]))
                            pl.plot(totalfitx, totalfity)
                            pl.plot(totalfitx, function3, 'r')
                            pl.savefig(newdir + wpsuffix + '_dataset-' + str(datalist[n][-7:-4]) + '_channel-' + str(SQs[j])+  '.png')
                            #print(len(jfe_channels))
                            #print(jfe_channels)
                            #print(len(maxvalsx[j]))
                            #if jfe_channels[i] != 0:
                                #newdirjfe = os.path.join(writepath+'potentialjfe/')
                                #if not os.path.exists(newdirjfe):
                                    #os.mkdir(newdirjfe)
                                #pl.savefig(newdirjfe + wpsuffix + '_dataset-' + str(datalist[n][-7:-4]) + '_channel-' + str(SQs[j])+  '.png')
                                
                            
                            
                        
                        
                        #pl.figure(1)  
                        #pl.plot(xLn0, yderivdetr)
                        #pl.plot(mvalsx, mvalsy, 'o')
                        #print(mvalsx)
                        #print(mvalsy)
                        #pl.show(block=False)
                        #pl.figure(2)
                        
                        #pl.plot(xLn0, yList0det)
                        
                        #pl.plot(totalfitx, totalfity)
                        #pl.plot(totalfitx, function3, 'r')
                        #pl.plot(xfit, ffitexp, 'g')
                        #pl.plot(mvalsx, mvalsy, 'o')
                        #pl.show()
                        #pl.clf()
                        
                        #pl.figure()
                        #pl.title('dataset: ' + str(datalist[n][-7:-4]) + ', channel: ' + str(SQs[j]))
                        #pl.plot(np.log(totalfitx), np.log(totalfity))
                        #pl.plot(np.log(totalfitx), np.log(function3), 'r')
                        #pl.plot(np.log(totalfitx), np.log(ffit_p_total), 'r')
                        #pl.plot(totalfitx, totalfity)
                        #pl.plot(totalfitx, function3, 'r')
                        #pl.plot(totalfitx, ffit_p_total, 'r')
                        #pl.pause(0.05)
                        #pl.show(block=False)
                        #print(coefs_p[0])
                        
                        #pl.draw()
               # pl.show()
            #pl.plot(totalfitx[10:], function3[10:], 'r')

            #pl.plot(totalfitx, ffitexp_totalmod, 'g')
            #pl.plot(totalfitx[:np.argmax(function1)+10], function1[:np.argmax(function1)+10], 'r')

            #pl.plot(totalfitx[np.argmax(function3)], function2[np.argmax(function3)], 'o', 'g')
            
            
                
            #pl.plot(totalfitx, ffit_re_total, 'r')
            #pl.plot(totalfitx, function, 'g')

            #pl.plot(xfit, ffitexp, 'r')
            #pl.plot(fitlistx_re_fit, ffit_re, 'r')

            #pl.plot(xfit, ffitexpmod, color='red')


                
                 
                    
    
            #tab for j for loop - loop over channels
        #tab for n for loop - loop over datasets
    #tab for final execute - outside of dataset loop
        
        
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

        #Coincidence finding! 
        if coincidence:
            meantcs = []
            meantimeconst = []
            for j in range(len(SQs)):
                meantcs.append(np.mean(timeconsts[j]))
            meantimeconst = np.mean(meantcs)

            if len(SQs)==3:
                coinc01 = []
                coinc01_time = []
                coinc02 = []
                coinc02_time = []
                coinc12 = []
                coinc12_time = []
                coinc012 = []
                #coinc012_time = []


                for i in range(len(peakvals[0])):
                    for j in range(len(peakvals[1])):
                        if np.abs(peakvals[0][i] - peakvals[1][j]) < meantimeconst*3:
                            coinc01.append(datalist[n][-7:-4])
                            coinc01_time.append(peakvals[0][i] - peakvals[1][j])

                for i in range(len(peakvals[0])):
                    for j in range(len(peakvals[2])):
                        if np.abs(peakvals[0][i] - peakvals[2][j]) < meantimeconst*3:
                            coinc02.append(datalist[n][-7:-4])
                            coinc02_time.append(peakvals[0][i] - peakvals[2][j])

                for i in range(len(peakvals[1])):
                    for j in range(len(peakvals[2])):
                        if np.abs(peakvals[1][i] - peakvals[2][j]) < meantimeconst*3:
                            coinc12.append(datalist[n][-7:-4])
                            coinc12_time.append(peakvals[1][i] - peakvals[2][j])

    #             for i in range(len(peakvals[0])):
    #                 for j in range(len(peakvals[1])):
    #                     for k in range(len(peakvals[2])):
    #                         if np.abs(peakvals[0][i] - peakvals[1][j]) < meantimeconst*3 and np.abs(peakvals[0][i] - peakvals[2][k]) < meantimeconst*3 and np.abs(peakvals[1][i] - peakvals[2][k]) < meantimeconst*3:
    #                             coinc012.append(datalist[n])#[-7:-4])

                allcoinc = [coinc01, coinc01_time, coinc02, coinc02_time, coinc12, coinc12_time, coinc012]
                print(allcoinc)
        ratesall = [sum(rates[0])/(len(datalist)*scantime), sum(rates[1])/(len(datalist)*scantime), sum(rates[2])/(len(datalist)*scantime)]
    
    
    ########################figuring out pickle appending
        
        
        #writepath = '/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/'
        #wpsuffix = '1-500_20210728'
        #if n == 0:
            #testarray = [[1,2]]
            #with open(writepath + 'testpickle' + wpsuffix, 'wb') as fp1:
                #pickle.dump(testarray, fp1)
            #print(testarray)
        #if n == 1:
            #testarray = [3,4]
            #print(testarray)
        ##if n > 0:
            #with open(writepath + 'testpickle' + wpsuffix, 'rb') as fp1:
                #testarrayload = pickle.load(fp1)
            #print(testarrayload)
            #testarrayload.append(testarray)
            #with open(writepath + 'testpickle' + wpsuffix, 'wb') as fp1:
                #pickle.dump(testarrayload, fp1)
            #print(testarrayload)
            
        #if n == 0:
            #with open(writepath + 'testpickle' + wpsuffix, 'wb') as fp1:
                #pickle.dump(energies_data, fp1)
        #else:
            #with open(writepath + 'testpickle' + wpsuffix, 'rb') as fp1:
                #energies_data_load = pickle.load(fp1)
            #energies_data_load.append(energies_data)
            #with open(writepath + 'testpickle' + wpsuffix, 'wb') as fp1:
                #pickle.dump(testarrayload, fp1)
         
        #figuring out memory dump
        #del my_array
        #del my_object
        #gc.collect()
    
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    
    
       
    # Plotting
        
        #if len(maxsy) > 0:
        #print("removed %i jumped flux events" %len(xdisc))        
        #pl.plot(xLn0, yderivdeter)
        #pl.plot(xLn0, yList0, 'r')
        #pl.figure(2)
        #pl.plot(maxvalsx[0], maxvalsy[0], 'o', color='r')
        #pl.plot(maxvalsx[1], maxvalsy[1], 'o', color='b')
        #pl.plot(maxvalsx[2], maxvalsy[2], 'o', color='g')
        #pl.xlabel("Time (s)")
        #pl.ylabel("Energy (keV)")
        #pl.title(datalist[n][:8] + datalist[n][-8:-4])
        #pl.plot(xLn0, ymeansub[2], 'b')
        #pl.plot(xLn0, ymeansub[1], 'g')
        #pl.plot(xLn0, yList0)
        #pl.show()
        #pl.plot(xLn0, notchyms[j], 'r')
        #pl.plot(xLn0, signal.medfilt(ymeansub[0]), 'r')
        #pl.plot(xLn0,signal.medfilt(ymeansub[1]), 'b')
        #pl.plot(xLn0,signal.medfilt(ymeansub[2]), 'g')
        #pl.show()
        #pl.plot(xLn0,signal.medfilt(ymeansub[2], 19), 'r')
        
        
        
        
        jfe_datasets.append(jfe_channels)
    jfe_array = np.array(jfe_datasets)
    totaljfe = np.sum(jfe_array)
    #print(np.sum(jfe_array, axis=0))
    #print(np.sum(jfe_array, axis=0)[2])
    #print(np.sum(jfe_array))
        
        
        
        #del dList0, yList0,
        #gc.collect()
    
    
    
    #print(timeconsts[0])
    #print(timeconsts[1])
    #print(timeconsts[2])
    bm = 3
    #num_bins1 = np.logspace(np.log10(0.1),np.log10(100),bm*25)
    num_bins1 = bm*25
    pl.figure()
    pl.xlabel("Energy, Chip 1 (floating), Control bolo")
    n1, bins1, patchs1 = pl.hist(energies[0], bins='auto')
    pl.xlabel("Energy, Chip 2 (heat sunk), Control bolo")
    n2, bins2, patchs2 = pl.hist(energies[1], bins='auto')
    pl.xlabel("Energy, Chip 1 (floating), Bling")
    n3, bins3, patchs3 = pl.hist(energies[2], bins='auto')
    #pl.show(block=False)
    #pl.show()
    #pl.clf()
    pl.figure()
    pl.xlabel("time (s), Chip 1 (floating), Control bolo")
    n1, bins1, patchs1 = pl.hist(timeconsts[0], bins='auto')
    pl.xlabel("time (s), Chip 2 (heat sunk), Control bolo")
    n2, bins2, patchs2 = pl.hist(timeconsts[1], bins='auto')
    pl.xlabel("time (s), Chip 1 (floating), Bling")
    n3, bins3, patchs3 = pl.hist(timeconsts[2], bins='auto')
    
    #pl.show()
    pl.clf()
    #print(jfe_datasets)
    TRE = np.sum(jfe_datasets)
    #print("Total rejected events = %i"%TRE)
    #print("Rate with hits rejected = ")
    #print("Rate without hits rejected = ")
    #print("total events found:", len(energies[0])+len(energies[1])+len(energies[2]))
    #print("channel 1 rate (Hz):", sum(rates[0])/(len(datalist)*scantime))
    #print("channel 2 rate (Hz):", sum(rates[1])/(len(datalist)*scantime))
    #print("channel 6 rate (Hz):", sum(rates[2])/(len(datalist)*scantime))
    
    #print(os.getcwd())
    #print("energies")
    #print(energies)
    #print("time constants")
    #print(timeconsts)
    #print(peakvals)
    #print(energies_data)
    numerrors = sum(errors)
    #global writepath
    
    

        
    #print(writepath + '_energies_' + wpsuffix)  
    with open(writepath + 'rates_' + wpsuffix, 'wb') as fp1:
        pickle.dump(ratesall, fp1)
    #print("ratesall:", ratesall)
    
    with open(writepath + 'energies_' + wpsuffix, 'wb') as fp2:
        pickle.dump(energies, fp2)
    #print("energies:", energies)
        
    with open(writepath + 'energies-data_' + wpsuffix, 'wb') as fp3:
        pickle.dump(energies_data, fp3)
    #print("energies_data:", energies_data)
           
    with open(writepath + 'timeconsts_' + wpsuffix, 'wb') as fp4:
        pickle.dump(timeconsts, fp4)
    #print("timeconsts:", timeconsts)
        
    with open(writepath + 'peakvals_' + wpsuffix, 'wb') as fp5:
        pickle.dump(peakvals, fp5)
    #print("peakvals:", peakvals)
        
    if coincidence:
        with open(writepath + 'coincidence_' + wpsuffix, 'wb') as fp6:
            pickle.dump(allcoinc, fp6)
        print("allcoinc:", allcoinc)
    
    with open(writepath + 'jfedatasets_' + wpsuffix, 'wb') as fp7:
        pickle.dump(jfe_datasets, fp7)
    #print("jfe_datasets:", jfe_datasets)
        
    with open(writepath + 'errors_' + wpsuffix, 'wb') as fp8:
        pickle.dump(numerrors, fp8)
    print("numerrors:", numerrors)
        
    
    if savefigs:
        
        transp = 0.75
        # energies from fits
        pl.clf() 
        n1, bins1, patchs1 = pl.hist(energies[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
        n2, bins2, patchs2 = pl.hist(energies[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
        n3, bins3, patchs3 = pl.hist(energies[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
        #pl.xscale('log')
        #pl.xlim(xmin = 0., xmax =200.)
        pl.title(wpsuffix)
        pl.xlabel("Pulse energy (arb. units)")
        pl.legend(loc='upper right')
        #pl.show()
        pl.savefig(writepath + 'energiesfromfits_' + wpsuffix + '.png')
        
        # energies from data
        pl.clf() 
        n1, bins1, patchs1 = pl.hist(energies_data[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
        n2, bins2, patchs2 = pl.hist(energies_data[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
        n3, bins3, patchs3 = pl.hist(energies_data[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
        #pl.xscale('log')
        #pl.xlim(xmin = 0., xmax =200.)
        pl.title(wpsuffix)
        pl.xlabel("Pulse energy (arb. units)")
        pl.legend(loc='upper right')
        #pl.show()
        pl.savefig(writepath + 'energiesfromdata_' + wpsuffix + '.png')
        
        # timeconstants
        pl.clf() 
        n1, bins1, patchs1 = pl.hist(timeconsts[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
        n2, bins2, patchs2 = pl.hist(timeconsts[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
        n3, bins3, patchs3 = pl.hist(timeconsts[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
        #pl.xscale('log')
        #pl.xlim(xmin = 0., xmax =200.)
        pl.title(wpsuffix)
        pl.xlabel("Pulse energy (arb. units)")
        pl.legend(loc='upper right')
        #pl.show()
        pl.savefig(writepath + 'timeconstants_' + wpsuffix + '.png')
        
        
        # writing values of interest to a text file
        txtfile = open(writepath + 'vals3_' + wpsuffix + '.txt', 'w')
        
        txtfile.write("Total number of events in " + str((len(datalist)*scantime)/3600.) + " hours of data: " + str(len(energies[0]+energies[1]+energies[2])) + " \n \n \n")
        
        txtfile.write("Jumped flux events: \n")
        txtfile.write("Total number of JFEs in bolo 1: "+ str(np.sum(jfe_array, axis=0)[0]) + " \n")
        txtfile.write("Total number of JFEs in bolo 2: "+ str(np.sum(jfe_array, axis=0)[1]) + " \n")
        txtfile.write("Total number of JFEs in bolo 3: "+ str(np.sum(jfe_array, axis=0)[2]) + " \n")
        txtfile.write("Total number of JFEs in all detectors: "+ str(np.sum(jfe_array)) + " \n")
        if len(energies[0]+energies[1]+energies[2]) != 0:
            txtfile.write("% of total events that were jumped flux: "+ str(100.*totaljfe/len(energies[0]+energies[1]+energies[2])) + "% \n \n \n")
        else:
            txtfile.write("not enough data \n \n \n")
            
        txtfile.write("Rates \n")
        txtfile.write("bolo 1 rate: " + str(ratesall[0]) + " Hz \n")
        txtfile.write("bolo 2 rate: " + str(ratesall[1]) + " Hz \n")
        txtfile.write("bolo 3 rate: " + str(ratesall[2]) + " Hz \n")
        if len(ratesall) != 0:
            txtfile.write("Average rate of all three channels: " + str(sum(ratesall)/len(ratesall)) + " Hz \n \n \n")
        else:
            txtfile.write("not enough data \n \n \n")
        
        
        txtfile.write("Time constants \n")
        txtfile.write("Mean time constant in bolo 1: " + str(timeconsts[0]) + " s \n")
        txtfile.write("Mean time constant in bolo 2: " + str(timeconsts[1]) + " s \n")
        txtfile.write("Mean time constant in bolo 3: " + str(timeconsts[2]) + " s \n")
        
        if len(timeconsts[0]+timeconsts[1]+timeconsts[2]) != 0:
            txtfile.write("Average time constant in all three channels: " + str(sum(timeconsts[0]+timeconsts[1]+timeconsts[2])/(len(timeconsts[0]+timeconsts[1]+timeconsts[2]))) + " seconds \n \n \n")
        else:
            txtfile.write("not enough data \n \n \n")
            
        if coincidence:
            txtfile.write("Number of double coincidence events:" + str(len(allcoinc[0])+len(allcoinc[2])+len(allcoinc[4]))+ "\n")
            txtfile.write("Number of triple coincidence events: N/A \n \n \n")
        
        
        
        txtfile.write("Number of fit errors: \n")
        txtfile.close

Main function


In [6]:

#print("Running 2021_03_19 - bolo 1: heatsunk control, bolo 2: floating control, bolo 3: floating bling, no source")
#CR_analyze(["/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_03_19/CR_Scan_023/20210319_SQ1-contr1_SQ2-contr2_SQ6-contr3_biasthreeturnsintotrans_5-FTR-5min_"], \
           #[44,999], [1,2,6], [100.,100.,100.], [0.34,0.365,0.367], 300., [True, True, True], ["20210319_b1-hscontr_b2-fltcontr_b3-fltblng_nosource_44-999"], showfits=False, noisefilt=False, verbose = False, fit = True, pwrcnv = True)

#print("Running 2021_03_23 - bolo 1: heatsunk control, bolo 2: floating control, bolo 3: floating bling, source")
#CR_analyze(["/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_03_23/CR_Scan_002/20210319_SQ1-contr1_SQ2-contr2_SQ6-contr3_biasthreeturnsintotrans_5-FTR-5min_Co60_"], \
          #[1,847], [1,2,6], [100.,100.,100.], [0.34,0.365,0.367], 300., [True, True, True],["20210323_b1-hscontr_b2-fltcontr_b3-fltblng_Co60_1-847"], showfits=False, noisefilt=False, verbose = False, fit = True, pwrcnv = True)

print("Running 20210516 - control bolos, no source")
CR_analyze(["/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_16-control_nosource/CR_Scan_002/_"], \
           [2,259], [1,2,6], [100.,100.,100.], [3.457,3.187,3.165], 300., [False, False, True], ["20210516_controlbolos_nosource_2-256"], showfits=False, noisefilt=False, verbose = False, fit = True, pwrcnv = True)

print("Running 20210517 - control bolos, source")
CR_analyze(["/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/source_"], \
           [1,824], [1,2,6], [100.,100.,100.], [3.517,3.164,3.150], 300., [False, False, True], ["20210517_controlbolos_Co-60_1-824"], showfits=False, noisefilt=False, verbose = False, fit = True, pwrcnv = True)

print("Running 20210528 - mitigated bolos, source")
CR_analyze(["/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/_"], \
           [1,727], [1,2,6], [100.,100.,100.], [2.809,2.911,1.48], 300., [False, False, True], ["20210528_controlbolos_Co-60_1-727"], showfits=False, noisefilt=False, verbose = False, fit = True, pwrcnv = True)

print("Running 20210604 - mitigated bolos, no source")
CR_analyze(["/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_06_04-mitig_nosource/CR_Scan_002/BT10-02-CR-with-mitigation-no-source_"], \
           [1,999], [1,2,6], [100.,100.,100.], [2.809,2.911,1.48], 300., [False, False, True], ["20210604_controlbolos_nosource_1-999"], showfits=False, noisefilt=False, verbose = False, fit = True, pwrcnv = True)













Running 20210516 - control bolos, no source
running data set 002
channel 1
channel 2
channel 6
running data set 003
channel 1
channel 2
channel 6
running data set 004
channel 1
channel 2
channel 6
running data set 005
channel 1
channel 2
channel 6
running data set 006
channel 1
channel 2
channel 6
running data set 007
channel 1
channel 2
channel 6
running data set 008
channel 1
channel 2
channel 6
running data set 009
channel 1
channel 2
channel 6
running data set 010
channel 1
channel 2
channel 6
running data set 011
channel 1
channel 2
channel 6
running data set 012
channel 1
channel 2
channel 6
running data set 013
channel 1
channel 2
channel 6
running data set 014
channel 1
channel 2
channel 6
running data set 015
channel 1
channel 2
channel 6
running data set 016
channel 1
channel 2
channel 6
running data set 017
channel 1
channel 2
channel 6
running data set 018
channel 1
channel 2
channel 6
running data set 019
channel 1
channel 2
channel 6
running data set 020
channel 1
channel

/Users/shawn/Library/Python/3.8/lib/python/site-packages/numpy/polynomial/polynomial.py:1361: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


channel 2
channel 6
running data set 029
channel 1
channel 2
channel 6
running data set 030
channel 1
channel 2
channel 6
running data set 031
channel 1
channel 2
channel 6
running data set 032
channel 1
channel 2
channel 6
running data set 033
channel 1
channel 2
channel 6
running data set 034
channel 1
channel 2
channel 6
running data set 035
channel 1
channel 2
channel 6
running data set 036
channel 1
channel 2
channel 6
running data set 037
channel 1


/Users/shawn/Library/Python/3.8/lib/python/site-packages/numpy/polynomial/polynomial.py:1361: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


channel 2


...

channel 6
running data set 856
channel 1
channel 2
channel 6
running data set 857
channel 1
channel 2
channel 6
running data set 858
channel 1
channel 2
channel 6
running data set 859
channel 1
channel 2
channel 6
running data set 860
channel 1
channel 2
channel 6
running data set 861
channel 1
channel 2
channel 6
running data set 862
channel 1
channel 2
channel 6
running data set 863
channel 1
channel 2
channel 6
running data set 864
channel 1
channel 2
channel 6
running data set 865
channel 1
channel 2
channel 6
running data set 866
channel 1
channel 2
channel 6
running data set 867
channel 1
channel 2
channel 6
running data set 868
channel 1
channel 2
channel 6
running data set 869
channel 1
channel 2
channel 6
running data set 870
channel 1
channel 2
channel 6
running data set 871
channel 1
channel 2
channel 6
running data set 872
channel 1
channel 2
channel 6
running data set 873
channel 1
channel 2
channel 6
running data set 874
channel 1
channel 2
channel 6
running data set 875


/Users/shawn/Library/Python/3.8/lib/python/site-packages/numpy/polynomial/polynomial.py:1361: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


failed fit found, correcting
channel 6
running data set 904
channel 1
channel 2
channel 6
running data set 905
channel 1
channel 2
channel 6
running data set 906
channel 1
channel 2
channel 6
running data set 907
channel 1
channel 2


/Users/shawn/Library/Python/3.8/lib/python/site-packages/numpy/polynomial/polynomial.py:1361: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


channel 6
running data set 908
channel 1
channel 2
channel 6
running data set 909
channel 1
channel 2
channel 6
running data set 910
channel 1
channel 2
channel 6
running data set 911
channel 1
channel 2
channel 6
running data set 912
channel 1
channel 2
channel 6
running data set 913
channel 1
channel 2
channel 6
running data set 914
channel 1
channel 2
channel 6
running data set 915
channel 1
channel 2
channel 6
running data set 916
channel 1
channel 2
channel 6
running data set 917
channel 1
channel 2
channel 6
running data set 918
channel 1
channel 2
channel 6
running data set 919
channel 1
channel 2
channel 6
running data set 920
channel 1
channel 2
channel 6
running data set 921
channel 1
channel 2
channel 6
running data set 922
channel 1
channel 2
channel 6
running data set 923
channel 1
channel 2
channel 6
running data set 924
channel 1
channel 2
channel 6
running data set 925
channel 1
channel 2
channel 6
running data set 926
channel 1
channel 2
channel 6
running data set 927
